In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
from glob import glob
import shutil
import random

import csv

In [2]:
device_dict={"S":"Smartphone", 
             "T":"Tablet",
             "L":"Laptop",
             "V":"VehicleLCD",
             "M":"Monitor"}
status_dict={"F":"Focus"}

In [3]:
class Scenario():
    def __init__(self):
        self.scenarios={'F':[self.s0,self.s1],
                        'S':[self.s4],
                        'D':[self.s5,self.s6],
                        'A':[self.s4],
                        'N':[self.s2,self.s3]}
        
    def set_points(self, nframes):
        self.nframes = nframes
        self.x0 = np.linspace(0, 1920, self.nframes+2)[1:-1]
        self.y0 = np.linspace(0, 1080, self.nframes+2)[1:-1]
        
        self.xround = np.concatenate((self.x0[::2],self.x0[-1::-2]))
        self.yround = np.concatenate((self.y0[::2],self.y0[-1::-2]))

    def xrand(self):
        return np.repeat(480 + 960*np.random.rand(), self.nframes)
        
    def yrand(self):
        return np.repeat(270 + 540*np.random.rand(), self.nframes)

        
    def xrand2(self):
        return np.repeat(480*np.random.rand() + \
                         int(np.random.rand())*480*3,
                         self.nframes)
    def yrand2(self):
        return np.repeat(270*np.random.rand() + \
                         int(np.random.rand())*270*3,
                         self.nframes)

    def s0(self): 
        return [self.x0,self.y0]
    def s1(self): 
        return [np.repeat(30, self.nframes), self.y0]
    def s2(self): 
        return [self.xround, self.y0]
    def s3(self): 
        return[self.x0, self.yround]
    def s4(self):
        return [self.xrand(), self.yrand()]

    def s5(self): 
        return [np.concatenate((self.xrand()[::5],self.xrand2()[::5],
                          self.xrand()[::5],self.xrand2()[::5],
                          self.xrand()[::5])), self.yrand2()]
    def s6(self):
        return [np.concatenate((self.yrand()[::5],self.yrand2()[::5],
                          self.yrand()[::5],self.yrand2()[::5],
                          self.yrand()[::5])), self.xrand2()]

In [5]:
fix_gaze = False # GazeAngle1을 CamAngle로 "복사"

for date in [2,4,5,8,9,10,11,12,16,17,18,19,22,23,25][5:]:
    #flist = glob(f"/media/di/mx500_2tb2/NIA2022_2/08{date:02d}/processing/S1/*/T1/*/RGB/*.mp4")
    flist = glob(f"/home/di/Work/data/NIA2022/08{date:02d}/processing/S1/*/T1/*/RGB/*.mp4")
    flist.sort()

    sco = Scenario()
    for fn in flist:
    #if True:
        out_dir = fn.split("NIA22EYE")[0]
        fn_ = fn.split("/")[-1]

        cap = cv2.VideoCapture(fn)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print( length )
        sco.set_points(length)

        _, _, ID, _, scenario, device, imgtype, status, action, orientation = fn_.split("_")
        orientation = orientation.split(".mp4")[0]

        #### Eye-tracker
        new_dir = out_dir.replace("RGB/", "Eye-tracker/")
        if not os.path.isdir(new_dir):
            os.mkdir(new_dir)

        # Knee -> lapTop
        #fn_ = fn_.replace("_K_", "_T_")

        fn_out = fn_.replace("rgb", "point")
        fn_out = fn_out.replace(".mp4", ".csv")

        with open(new_dir+fn_out, "w") as f:
            f.write("[point] x y \n")
            px, py = random.choice(sco.scenarios[status])()
            for xx, yy in zip(px, py):
                f.write(f"{int(xx)}, {int(yy)}\n")

            print(new_dir+fn_out, "done")

    

328
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Laptop/Eye-tracker/NIA22EYE_S1_013_T1_S03_L_point_A_C_T.csv done
327
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Laptop/Eye-tracker/NIA22EYE_S1_013_T1_S03_L_point_A_D_T.csv done
327
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Laptop/Eye-tracker/NIA22EYE_S1_013_T1_S03_L_point_A_E_T.csv done
327
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Laptop/Eye-tracker/NIA22EYE_S1_013_T1_S03_L_point_A_F_T.csv done
328
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Laptop/Eye-tracker/NIA22EYE_S1_013_T1_S03_L_point_A_T_T.csv done
328
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Laptop/Eye-tracker/NIA22EYE_S1_013_T1_S03_L_point_A_U_T.csv done
328
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Laptop/Eye-tracker/NIA22EYE_S1_013_T1_S03_L_point_D_C_T.csv done
327
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Laptop/Eye-tracker/NIA22EYE_S1_013_T1_S03_L_point_D_D_T.csv done
328
/home/di/Work/data/N

300
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Monitor/Eye-tracker/NIA22EYE_S1_013_T1_S05_M_point_N_U_T.csv done
300
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Monitor/Eye-tracker/NIA22EYE_S1_013_T1_S05_M_point_S_C_T.csv done
300
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Monitor/Eye-tracker/NIA22EYE_S1_013_T1_S05_M_point_S_D_T.csv done
300
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Monitor/Eye-tracker/NIA22EYE_S1_013_T1_S05_M_point_S_E_T.csv done
310
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_D_C_L.csv done
312
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_D_C_R.csv done
310
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_D_C_T.csv done
305
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_D_D_L.csv done
306


304
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_N_S_L.csv done
306
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_N_S_R.csv done
304
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_N_S_T.csv done
305
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_N_T_L.csv done
306
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_N_T_R.csv done
305
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S02_S_point_N_T_T.csv done
309
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S10_S_point_A_C_L.csv done
308
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Smartphone/Eye-tracker/NIA22EYE_S1_013_T1_S10_S_point_A_C_R.c

/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S02_T_point_D_U_T.csv done
309
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S02_T_point_F_C_L.csv done
309
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S02_T_point_F_C_R.csv done
309
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S02_T_point_F_C_T.csv done
302
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S02_T_point_F_D_L.csv done
307
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S02_T_point_F_D_R.csv done
301
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S02_T_point_F_D_T.csv done
308
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S02_T_point_F_E_L.csv done
303
/home/di/Work/data/NIA20

299
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S10_T_point_A_P_R.csv done
302
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S10_T_point_A_P_T.csv done
300
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S10_T_point_A_S_L.csv done
302
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S10_T_point_A_S_R.csv done
301
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S10_T_point_A_S_T.csv done
305
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S10_T_point_A_T_L.csv done
306
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S10_T_point_A_T_R.csv done
305
/home/di/Work/data/NIA2022/0810/processing/S1/013/T1/Tablet/Eye-tracker/NIA22EYE_S1_013_T1_S10_T_point_A_T_T.csv done
311
/home/di/Work/data/N

...

328
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Laptop/Eye-tracker/NIA22EYE_S1_032_T1_S02_L_point_N_U_T.csv done
329
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Laptop/Eye-tracker/NIA22EYE_S1_032_T1_S02_L_point_S_C_T.csv done
327
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Laptop/Eye-tracker/NIA22EYE_S1_032_T1_S02_L_point_S_D_T.csv done
329
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Laptop/Eye-tracker/NIA22EYE_S1_032_T1_S02_L_point_S_E_T.csv done
327
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Laptop/Eye-tracker/NIA22EYE_S1_032_T1_S02_L_point_S_F_T.csv done
327
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Laptop/Eye-tracker/NIA22EYE_S1_032_T1_S02_L_point_S_U_T.csv done
329
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Laptop/Eye-tracker/NIA22EYE_S1_032_T1_S03_L_point_A_C_T.csv done
328
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Laptop/Eye-tracker/NIA22EYE_S1_032_T1_S03_L_point_A_D_T.csv done
329
/home/di/Work/data/N

311
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S01_S_point_D_C_R.csv done
309
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S01_S_point_D_C_T.csv done
306
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S01_S_point_D_D_L.csv done
305
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S01_S_point_D_D_R.csv done
307
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S01_S_point_D_D_T.csv done
307
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S01_S_point_D_E_L.csv done
305
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S01_S_point_D_E_R.csv done
306
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S01_S_point_D_E_T.c

308
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S10_S_point_A_E_R.csv done
306
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S10_S_point_A_E_T.csv done
308
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S10_S_point_A_F_L.csv done
309
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S10_S_point_A_F_R.csv done
305
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S10_S_point_A_F_T.csv done
306
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S10_S_point_A_L_L.csv done
309
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S10_S_point_A_L_R.csv done
307
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Smartphone/Eye-tracker/NIA22EYE_S1_032_T1_S10_S_point_A_L_T.c

301
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S01_T_point_F_F_R.csv done
299
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S01_T_point_F_F_T.csv done
305
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S01_T_point_F_H_L.csv done
309
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S01_T_point_F_H_R.csv done
306
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S01_T_point_F_H_T.csv done
308
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S01_T_point_F_L_L.csv done
305
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S01_T_point_F_L_R.csv done
309
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S01_T_point_F_L_T.csv done
302
/home/di/Work/data/N

301
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S10_T_point_S_D_L.csv done
301
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S10_T_point_S_D_R.csv done
301
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S10_T_point_S_D_T.csv done
308
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S10_T_point_S_E_L.csv done
311
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S10_T_point_S_E_R.csv done
309
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S10_T_point_S_E_T.csv done
302
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S10_T_point_S_F_L.csv done
302
/home/di/Work/data/NIA2022/0825/processing/S1/032/T1/Tablet/Eye-tracker/NIA22EYE_S1_032_T1_S10_T_point_S_F_R.csv done
300
/home/di/Work/data/N

0906 이후 버전에선 필요 없음. 

In [71]:
    #### fix gaze -> Cam
    if fix_gaze:
        old_cam_dir = out_dir.replace("RGB/", "GazeAngle1/")
        new_cam_dir = out_dir.replace("RGB/", "CamAngle/")
        if not os.path.isdir(new_cam_dir):
            shutil.copytree(old_cam_dir, new_cam_dir)

        old_fn_ = fn_.replace("rgb", "gaze1")
        old_fn_ = old_fn_.replace(".mp4", ".txt") # txt or csv?
        new_fn_ = old_fn_.replace("gaze1", "cam")
        new_fn_ = new_fn_.replace("txt", "csv")
        # 앞, 뒤 컬럼 삭제
        try:
            fr = csv.reader(open(new_cam_dir+old_fn_, "r"))
            #fw = csv.writer(new_cam_dir+new_fn_, "w"):
            with open(new_cam_dir+new_fn_, "w") as fw:
                for lls in fr:
                    #lls = ll.split(",")
                    if len(lls) > 3:
                        fw.write(", ".join(lls[1:-1])+"\n")
        except:
            print("error", old_fn_)
        os.remove(new_cam_dir+old_fn_)

300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_033_T1_S05_V_point_F_D_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_033_T1_S05_V_point_D_C_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_033_T1_S04_V_point_S_E_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_033_T1_S03_V_point_D_D_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_033_T1_S04_V_point_N_E_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_033_T1_S03_V_point_S_D_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_033_T1_S05_V_point_S_C_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/VehicleLCD/Eye-tracker/NIA22E

311
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_F_C_R.csv done
306
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S10_S_point_A_F_T.csv done
305
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_F_F_R.csv done
304
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_D_L_L.csv done
310
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S10_S_point_S_C_L.csv done
306
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_N_E_R.csv done
306
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S10_S_point_S_D_R.csv done
305
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22E

304
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_N_L_T.csv done
306
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_D_S_T.csv done
307
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_D_F_R.csv done
309
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_D_C_L.csv done
307
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S10_S_point_A_T_L.csv done
304
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S02_S_point_F_T_L.csv done
306
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22EYE_S1_033_T1_S10_S_point_A_S_L.csv done
307
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Smartphone/Eye-tracker/NIA22E

/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Laptop/Eye-tracker/NIA22EYE_S1_033_T1_S05_L_point_D_D_T.csv done
328
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Laptop/Eye-tracker/NIA22EYE_S1_033_T1_S04_L_point_N_E_T.csv done
326
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Laptop/Eye-tracker/NIA22EYE_S1_033_T1_S05_L_point_N_F_T.csv done
329
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Laptop/Eye-tracker/NIA22EYE_S1_033_T1_S04_L_point_F_C_T.csv done
327
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Laptop/Eye-tracker/NIA22EYE_S1_033_T1_S05_L_point_F_D_T.csv done
330
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Laptop/Eye-tracker/NIA22EYE_S1_033_T1_S05_L_point_N_E_T.csv done
330
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Laptop/Eye-tracker/NIA22EYE_S1_033_T1_S03_L_point_S_D_T.csv done
327
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Laptop/Eye-tracker/NIA22EYE_S1_033_T1_S03_L_point_S_U_T.csv d

311
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S02_T_point_D_E_T.csv done
302
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S10_T_point_A_S_R.csv done
311
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S10_T_point_A_U_R.csv done
302
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S02_T_point_D_D_L.csv done
301
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S02_T_point_D_F_L.csv done
308
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S02_T_point_F_U_R.csv done
304
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S02_T_point_D_T_T.csv done
301
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S02_T_point_N_P_L.c

300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S10_T_point_S_F_R.csv done
303
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/033/T1/Tablet/Eye-tracker/NIA22EYE_S1_033_T1_S02_T_point_D_P_R.csv done
0
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_034_T1_S05_V_point_S_C_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_034_T1_S06_V_point_A_C_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_034_T1_S05_V_point_A_C_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_034_T1_S06_V_point_A_U_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_034_T1_S06_V_point_N_U_T.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/VehicleLCD/Eye-tracker/NIA22EYE_S1_034_

308
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S10_S_point_A_L_T.csv done
302
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S03_S_point_D_S_L.csv done
304
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S03_S_point_N_S_R.csv done
306
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S10_S_point_S_E_R.csv done
308
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S03_S_point_N_L_T.csv done
309
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S03_S_point_F_H_L.csv done
305
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S03_S_point_D_D_L.csv done
308
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22E

310
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S10_S_point_A_L_R.csv done
308
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S10_S_point_S_L_L.csv done
307
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S10_S_point_A_T_L.csv done
306
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S03_S_point_N_T_T.csv done
304
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S03_S_point_N_S_T.csv done
306
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S03_S_point_N_E_R.csv done
308
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22EYE_S1_034_T1_S10_S_point_A_L_L.csv done
305
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Smartphone/Eye-tracker/NIA22E

310
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S10_T_point_S_E_T.csv done
302
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_F_F_L.csv done
303
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_N_U_L.csv done
307
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S10_T_point_S_P_L.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S10_T_point_S_F_T.csv done
302
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S10_T_point_S_L_L.csv done
301
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S10_T_point_S_F_L.csv done
311
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_D_E_T.c

304
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_N_U_R.csv done
302
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_D_E_R.csv done
302
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_F_L_T.csv done
301
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_D_D_L.csv done
300
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_D_L_L.csv done
311
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_F_P_R.csv done
308
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S10_T_point_A_P_T.csv done
304
/media/di/mx500_2tb2/NIA2022_2/0826/processing/S1/034/T1/Tablet/Eye-tracker/NIA22EYE_S1_034_T1_S03_T_point_D_U_T.c